In [54]:
import json
from IPython.core.display import JSON
import re

In [2]:
with open('conference_browsable_entities_list.json','r') as fr:
    confs = json.load(fr)

In [3]:
JSON(confs)

<IPython.core.display.JSON object>

In [24]:
# OLD
# unique_confs = {}
# for list_confs in confs:
#     for conf, conf_name in list_confs.items():
#         unique_confs[conf_name] = conf

# NEW
unique_confs = {}
for conf, conf_id in confs.items():
    unique_confs[conf] = conf_id
    cleaned_name = re.sub(r'\([^)]*\)', '', conf).strip()
    if cleaned_name not in unique_confs:
        unique_confs[cleaned_name] = conf_id

print(len(unique_confs), len(confs))

1869 1583


In [25]:
unique_confs

{'International Conference on Applied Informatics': 'icai2',
 'International Conference on Educational and Information Technology': 'iceit',
 'Formalizing Natural Languages with NooJ and Its Natural Language Processing Applications': 'nooj',
 'Robotics, Automation and Mechatronics': 'ram',
 'Advanced Intelligent System and Informatics': 'aisi',
 'International Workshop on Big Data and Information Security': 'iwbis',
 'IEEE International Conference on Blockchain and Cryptocurrency': 'icbc2',
 'IEEE International Symposium on Workload Characterization': 'iiswc',
 'BIOSIG': 'biosig',
 'NASA Formal Methods': 'nfm',
 'International Reliability Physics Symposium': 'irps',
 'Software Engineering and Information Management': 'icsim',
 'DMS Conference on Visualization and Visual Languages': 'dms',
 'Analysis of Images, Social Networks and Texts - AIST-АИСТ': 'aist',
 'Computational Models of Argument': 'comma',
 'International Conference on Metrology for Extended Reality, Artificial Intelligenc

In [26]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = list(unique_confs.keys())

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

vector_size = embeddings.shape[1]


(1869, 384)


In [27]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(vector_size)   # build the index
print(index.is_trained)

True


In [28]:
index.add(embeddings)                  # add vectors to the index
print(index.ntotal)

1869


# Save index

In [55]:
to_save = dict()
to_save["sentences"] = sentences
to_save["index"] = index
to_save["confs"] = unique_confs
to_save["dblp"] = {v:k for k,v in confs.items()}
import pickle
with open('AIDA.pickle', 'wb') as handle:
    pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Test Index

In [50]:
sentence = "International Semantic Web Conference"
sentence = "24th International Semantic Web Conference"
sentence = "The Web Conference (WWW)"
sentence = "The European Conference on Information Retrieval"
sentence = "European Conference on Advances in Information Retrieval (ECIR)"
sentence = "European Conference on IR Research (ECIR)"
# sentence = "5th International Workshop on Scientific Knowledge: Representation, Discovery, and Assessment"
embedding = model.encode([sentence])

In [51]:
D, I = index.search(embedding, k=3)

In [52]:
D

array([[6.7437310e-13, 1.6942883e-01, 1.0244387e+00]], dtype=float32)

In [53]:
for i in I[0]:
    print(sentences[i])

European Conference on IR Research (ECIR)
European Conference on IR Research
EuroVR International Conference
